In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import ast

# make pandas show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [12]:
#reading dataset from parquet file
df = pd.read_parquet('../datasets/train.parquet')
df.head()

,date,userId,sessionId,pageType,itemId,category,productPrice,oldProductPrice
0,2019-08-05 19:30:37,00172f1d9a71e9a8de0aa34288a6b19b,e8167c23f8ac2f9be979c32380e0fc2b7e94941e917d30...,productDetail,83472aea4051c00d031b01ff42ef73fc,"[""kadın çanta"",""omuz askılı çanta""]",622.0,1220.0
1,2019-08-31 16:53:55,00172f1d9a71e9a8de0aa34288a6b19b,c7f54acdf56e2d7539ffa59107b9017c2a8164495df909...,category,[],"[""seyahat samsonite"",""laptop çantası""]",NaN,NaN
2,2019-08-31 16:53:29,00172f1d9a71e9a8de0aa34288a6b19b,c7f54acdf56e2d7539ffa59107b9017c2a8164495df909...,main,[],[],NaN,NaN
3,2019-08-31 16:53:43,00172f1d9a71e9a8de0aa34288a6b19b,c7f54acdf56e2d7539ffa59107b9017c2a8164495df909...,category,[],"[""seyahat samsonite"",""laptop çantası""]",NaN,NaN
4,2019-08-31 16:54:13,00172f1d9a71e9a8de0aa34288a6b19b,c7f54acdf56e2d7539ffa59107b9017c2a8164495df909...,productDetail,d6afa22ab475d41e7dc9b721f3f795ad,"[""seyahat samsonite"",""laptop çantası""]",389.0,389.0


# Null Kontrolu

In [13]:
df.userId.nunique()

30133

In [14]:
len(df)

691842

In [15]:
# null check
df.isnull().sum()

date                    0
userId                  0
sessionId               0
pageType                0
itemId                  0
category                0
productPrice       473857
oldProductPrice    473857
dtype: int64

In [16]:
df.pageType.value_counts()

pageType
category         344349
productDetail    287731
main              40367
cart              18790
success             605
Name: count, dtype: int64


productPrice ile oldProductPrice beraber bir sekildemi nulllar onu kontrol edecegim

In [17]:
# check if productPrice and oldProductPrice are null at the same time always (row wise)
productPrice_null_oldProductPrice_not_null = len(df[(df['productPrice'].isnull()) & (df['oldProductPrice'].notnull())])

# oldProductPrice null iken productPrice'ın null olmadığı durumlar
oldProductPrice_null_productPrice_not_null = len(df[(df['oldProductPrice'].isnull()) & (df['productPrice'].notnull())])

# İki durumun toplamı
total = productPrice_null_oldProductPrice_not_null + oldProductPrice_null_productPrice_not_null
print(total)


0


Evet productPrice ile oldProductPrice beraber bir sekilde nullar, yani biri null iken digeri dolu olmuyor. Bunlari doldurmak icin bir yol bulmamiz gerekiyor.

In [18]:
# pageType'a gore null check
print("productPrice Null Counts")
for pageType in df.pageType.unique():
    print(pageType, df[df.pageType == pageType]['productPrice'].isnull().sum())

productPrice Null Counts
productDetail 69746
category 344349
main 40367
cart 18790
success 605


fiyatlar category, main, cart kisminda null gelmesi normal oyle olmasi gerekiyor zaten ama productDetail kisminda null olmamasi gerekiyordu bunu kurtalabilir miyiz diye bakacagim.

Bunun icin urunun fiyati acaba bir onceki sessionlarda varmiydi ona bakacagim. Eger varsa onu kullanacagim yoksa null birakacagim.

In [19]:
# pageType'a gore null check
print("productPrice Null Counts")
for pageType in df.pageType.unique():
    print(pageType, df[df.pageType == pageType]['productPrice'].isnull().sum())

productPrice Null Counts
productDetail 69746
category 344349
main 40367
cart 18790
success 605


In [20]:


df['date'] = pd.to_datetime(df['date'])
# Veri setini itemId ve date bazında sıralama
df = df.sort_values(by=['itemId', 'date'])

# En yakın dolu değerle doldurma fonksiyonu
def fill_missing_with_nearest(df, column):
    dataframe = df.copy()
    for idx, row in dataframe.iterrows():
        if pd.isna(row[column]):
            # Aynı itemId içindeki dolu değerleri bul
            non_na_values = dataframe[~dataframe[column].isna()]
            if not non_na_values.empty:
                # En yakın tarihi bul
                nearest_idx = (non_na_values['date'] - row['date']).abs().idxmin()
                dataframe.at[idx, column] = dataframe.at[nearest_idx, column]
    return dataframe

# Her itemId için eksik değerleri doldurma
df = df.groupby('itemId').apply(lambda group: fill_missing_with_nearest(group, 'productPrice')).reset_index(drop=True)
df = df.groupby('itemId').apply(lambda group: fill_missing_with_nearest(group, 'oldProductPrice')).reset_index(drop=True)


/var/folders/vj/3b1wf7ys74l0y3v418_26h7h0000gn/T/ipykernel_7975/382220264.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('itemId').apply(lambda group: fill_missing_with_nearest(group, 'productPrice')).reset_index(drop=True)
/var/folders/vj/3b1wf7ys74l0y3v418_26h7h0000gn/T/ipykernel_7975/382220264.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('itemId').apply(lambd

Normalde ileriki tarihten veri alarak onceki veriyi doldurmak data leakage olurdu ama elimizdeki veri zaten gecmiste gerceklestigi icin boyle bir varsayimla data leakage'dan yirttigimi varsaydim :) yinede test setini ayirmam gerekiyor sonra null doldurma baslayabilir

tekrardan null kontrolu yapiyorum

In [21]:
# pageType'a gore null check
print("productPrice Null Counts")
for pageType in df.pageType.unique():
    print(pageType, df[df.pageType == pageType]['productPrice'].isnull().sum())

productPrice Null Counts
productDetail 2
cart 18790
success 605
category 344349
main 40367


productDetail'da 2 tane null var bu sayi 700.000 satirlik veride ihmal edilebilir oldugu icin dropluyorum

In [22]:
# pageType==productDetail ise ve null varsa dropla
df = df[~((df.pageType == 'productDetail') & (df.productPrice.isnull()))]

In [23]:
# pageType'a gore null check
print("productPrice Null Counts")
for pageType in df.pageType.unique():
    print(pageType, df[df.pageType == pageType]['productPrice'].isnull().sum())

productPrice Null Counts
productDetail 0
cart 18790
success 605
category 344349
main 40367


pagetype== productDetail haricinde zaten urunun fiyati null gelecekti modelin fiyat konusundaki agirligi ne olursa olsun fiyat 0 ile doldurursam carpim 0 olacagindan 0 ile doldurabilirim diye dusundum ancak success kismini doldurabilirim fakat orada bazen birden fazla urun oluyor ve bunun icin product price kismini listeye cevirmem gerekiyor, 605/700.000 satirlik veri icin bunu yapmak mantikli gelmedi o sebeple 0 ile doldurmayi tercih ettim.


In [24]:
df['productPrice'] = df['productPrice'].fillna(0)
df['oldProductPrice'] = df['oldProductPrice'].fillna(0)

In [25]:
df.isnull().sum()

date               0
userId             0
sessionId          0
pageType           0
itemId             0
category           0
productPrice       0
oldProductPrice    0
dtype: int64

artik null verimiz yok

In [26]:
df.head()

,date,userId,sessionId,pageType,itemId,category,productPrice,oldProductPrice
0,2019-08-03 21:12:27,9c88dc8a43a48e2fcc648467971a17c2,1c50b964c487bfbc17945c636ec26c9189bca2739b2ea5...,productDetail,0000e475c4e76a1f4a615f33bbba9e79,"[""kadın ayakkabi"",""günlük ayakkabı""]",169.0,558.0
1,2019-08-04 18:27:44,2d0686f5e78772987a66197305a22925,b4d9b6249354494000a847615eeb0cd8e99165a6e3ff0e...,productDetail,0000e475c4e76a1f4a615f33bbba9e79,"[""kadın ayakkabi"",""günlük ayakkabı""]",169.0,558.0
2,2019-08-04 21:33:40,9c88dc8a43a48e2fcc648467971a17c2,5ebb9403eba9e14b4cb5184da53542ad27e680da2bd687...,productDetail,0000e475c4e76a1f4a615f33bbba9e79,"[""kadın ayakkabi"",""günlük ayakkabı""]",169.0,558.0
3,2019-08-05 06:34:26,4c59bebe04fad7eae9c5235acdb3201a,687a6d09b1dcaf07dd20a47901cb46f7d938ee097e45bf...,productDetail,0000e475c4e76a1f4a615f33bbba9e79,"[""kadın ayakkabi"",""günlük ayakkabı""]",169.0,558.0
4,2019-08-05 06:34:30,4c59bebe04fad7eae9c5235acdb3201a,687a6d09b1dcaf07dd20a47901cb46f7d938ee097e45bf...,productDetail,0000e475c4e76a1f4a615f33bbba9e79,"[""kadın ayakkabi"",""günlük ayakkabı""]",169.0,558.0
